In [11]:
import numpy as np
from ipywidgets import interact, widgets
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras import optimizers
from keras import Model
import keras

import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap.umap_ as umap

from sklearn.metrics import classification_report

In [2]:
import zipfile
with zipfile.ZipFile("felidae.zip", 'r') as zip_ref:
  zip_ref.extractall(".")

In [3]:
images = []
species = []
folderin = os.listdir('./Felidae/')
for folder in folderin:
    dir = os.listdir('./Felidae/'+folder)
    for image in dir:
        species.append(image[:-8])
        image = Image.open('./Felidae/'+folder+'/'+image).convert('RGB')
        image = np.asarray(image.resize((200,200)))
        image = image/255
        images.append(image)

In [5]:
def showimage(imageno):
    plt.figure(figsize=(5,5))
    plt.imshow(images[imageno])

In [6]:
interact(showimage,
    imageno=widgets.IntSlider(min=0,max=219,step=1,value=10))

interactive(children=(IntSlider(value=10, description='imageno', max=219), Output()), _dom_classes=('widget-in…

<function __main__.showimage(imageno)>

In [7]:
uniquelabel = np.unique(species)

label_to_int = {}  

for i in range(len(uniquelabel)):
    replace = uniquelabel[i]
    label_to_int[replace] = i

print(label_to_int)

species_as_int = [label_to_int[label] for label in species]

print(species_as_int)

{'Cheetah': 0, 'Leopard': 1, 'Lion': 2, 'Puma': 3, 'Tiger': 4}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [8]:
one_hot_species = keras.utils.to_categorical(species_as_int, 5)
print(one_hot_species)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [9]:
images=np.array(images)
labels=np.array(one_hot_species)
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(194, 200, 200, 3)
(49, 200, 200, 3)
(194, 5)
(49, 5)


In [10]:
from skimage.feature import hog
from skimage.color import rgb2gray

def extract_hog_features(images):
    hog_features = []
    for img in images:
        img_gray = rgb2gray(img) if len(img.shape) == 3 and img.shape[2] == 3 else img
        features, _ = hog(img_gray, visualize=True, block_norm='L2-Hys', feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

In [12]:
reducer = umap.UMAP(n_components=100, random_state=42)
X_train_umap = reducer.fit_transform(X_train_hog)
X_test_umap = reducer.transform(X_test_hog)

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [14]:
from sklearn import cluster
from sklearn.metrics import silhouette_score

k = 3
seed = 99

km = cluster.KMeans(n_clusters=k, init='random', random_state=seed)
km.fit(X_train_umap)

C_km = km.predict(X_test_umap)

sil_score = silhouette_score(X_test_umap, C_km)

print("Cluster assignment:", C_km)
print("Silhouette Score:", sil_score)


Cluster assignment: [2 2 2 1 2 1 2 2 0 0 1 0 2 1 1 0 1 0 1 0 1 0 2 2 0 2 2 1 0 2 2 2 0 0 1 1 2
 2 2 2 1 1 2 0 2 2 1 0 1]
Silhouette Score: 0.22407308


c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
